# 유기동물 정보 가져와서 DB에 넣기

동물보호관리시스템 유기동물 조회 서비스:
<https://www.data.go.kr/dataset/15001096/openapi.do>

공공데이터포털 API를 사용하기 위해 환경변수를 설정합니다.

반드시 Key, Token 등의 내용을 바꿔서 입력해 주세요.

```bash
# Mac, Linux 또는 Windows Git Bash 등에선 다음과 같이 export를 사용하시면 됩니다.
export ANIMAL_API_KEY=abcdefghijklm%2FvSfZeu4FoGpl2eGP%2FzlR5eGh%2F9M8jbOv4HPFZI%2BM7ckZstS8OzkMKHNBkpAtdZX6T8VoRl4Q%3D%3D
# Windows 기본 환경에선 다음과 같이 set을 사용하세요.
set ANIMAL_API_KEY=abcdefghijklm%2FvSfZeu4FoGpl2eGP%2FzlR5eGh%2F9M8jbOv4HPFZI%2BM7ckZstS8OzkMKHNBkpAtdZX6T8VoRl4Q%3D%3D

# Jupyter Notebook 실행
jupyter notebook
```

환경 변수에서 API 키를 가져옵니다.

In [1]:
# 환경 변수에서 API 키 가져오기

import os
from urllib.parse import unquote

API_KEY = unquote(os.environ['ANIMAL_API_KEY'])

## 라이브러리 사용

In [2]:
# requests 사용

import requests

requests.__version__

'2.22.0'

In [3]:
# sqlite3 사용

import sqlite3

sqlite3.version

'2.6.0'

## 동물보호관리시스템 유기동물 조회 서비스 API 사용

DB에 꽉 채워넣읍시다.

In [4]:
# 동물보호관리시스템 유기동물 조회 서비스 API

# 물음표(?) 앞부분
url = 'http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic'

# 물음표(?) 뒷부분
payload = {
    'serviceKey': API_KEY,
    'bgnde': '20191001',
    'endde': '20191031',
    'numOfRows': 100,
}

# API 호출
response = requests.get(url, params=payload)

print(response)

<Response [200]>


In [10]:
# Response의 Text 길이를 확인합니다.
# 만약 이 값이 너무 작다면 오류가 발생했는지 확인해 보세요.

len(response.text)

78282

In [11]:
# XML 데이터 파싱

import xml.etree.ElementTree as ET

root = ET.fromstring(response.text)

root

<Element 'response' at 0x110b55890>

In [12]:
# 아이템 확인

item = next(root.iter('item'))

for child in item:
    print(child.tag, '-', child.text)

age - 2019(년생)
careAddr - 제주특별자치도 제주시 첨단동길 184-14 (용강동) 
careNm - 제주유기동물보호센터
careTel - 064-710-4067
chargeNm - 제주도임시
colorCd - 검정
desertionNo - 450650201906393
filename - http://www.animal.go.kr/files/shelter/2019/10/201910231410974_s.jpg
happenDt - 20191023
happenPlace - 서귀포시 안덕 환순로 56번길 26-3
kindCd - [개] 믹스견
neuterYn - N
noticeEdt - 20191104
noticeNo - 제주-제주-2019-06023
noticeSdt - 20191023
orgNm - 제주특별자치도
popfile - http://www.animal.go.kr/files/shelter/2019/10/201910231410974.jpg
processState - 보호중
sexCd - M
specialMark - (개체관리번호 6002)생후2개월
weight - 2(Kg)


In [13]:
# 컬럼 목록 얻기

item = next(root.iter('item'))

columns = []

for child in item:
    columns.append(child.tag)

columns

['age',
 'careAddr',
 'careNm',
 'careTel',
 'chargeNm',
 'colorCd',
 'desertionNo',
 'filename',
 'happenDt',
 'happenPlace',
 'kindCd',
 'neuterYn',
 'noticeEdt',
 'noticeNo',
 'noticeSdt',
 'orgNm',
 'popfile',
 'processState',
 'sexCd',
 'specialMark',
 'weight']

## 사용할 컬럼 선택

어떤 컬럼을 사용할지 선택해서 테이블을 만듭시다.

저는 `age`, `colorCd`, `happenDt`, `kindCd`, `orgNm`, `sexCd`, `weight`만 사용하겠습니다.

In [14]:
columns = ['age', 'colorCd', 'happenDt', 'kindCd', 'orgNm', 'sexCd', 'weight']

## Identifier (고유 식별자)

각 동물 데이터에 대한 Identifier를 만들면 개별적으로 관리하기 좋습니다.

학급에서 학생들에게 번호를 지정하면 같은 이름을 가진 학생을 구분하기도 좋고, 훨씬 쉽고 빠르게 데이터에 접근할 수 있습니다.

“아샬”이란 학생의 국어 점수를 찾으려면 꽤 어려울 수 있지만,
(학생 정보가 번호 순서대로 써있다는 전제로) 32번 학생의 국어 점수는 비교적 쉽게 찾을 수 있죠.

SQLite는 `PRIMARY_KEY`와 `AUTOINCREMENT`를 통해 Identifier를 제공합니다.

- [CREATE TABLE](https://www.sqlite.org/lang_createtable.html)
- [SQLite Primary Key](https://www.sqlitetutorial.net/sqlite-primary-key/)
- [SQLite AUTOINCREMENT](https://www.sqlitetutorial.net/sqlite-autoincrement/)

## Hash

- 해시 함수(hash function)는 임의의 길이의 데이터를 고정된 길이의 데이터로 매핑하는 함수이다. 
- 저장된 자료의 양에 상관없이 원소 하나를 저장·검색하는 데 항상 상수 시간에 가능하게 할 수 없는지 사람들은 요구하게 되었고, 이 꿈을 실현한 것이 해시 테이블이다. 해시 테이블은 자료의 저장·검색에 있어 극단적인 효율에 다다른 자료구조이다.

- 해시(Hash) 테이블은 원소의 값에 의해 결정되는 자료구조이다. 즉, 저장된 자료와의 비교를 통해 자리를 찾지 않고 단 한번의 계산으로 자리를 찾는다.


API에서 이미 Identifier를 제공하고 있다면 그걸 그대로 활용할 수 있습니다.

유기동물 조회 서비스 API는 `noticeNo` 항목으로 “`ㅇㅇ-ㅇㅇ-2019-00000`” 같은 값을 제공하는데,
이건 숫자가 아니라서 우리의 DB에 Identifier로 바로 활용할 수 없습니다.

그래서 Python은 `hash` 함수를 제공합니다.

In [15]:
# 문자열을 숫자로 변환

hash('ㅇㅇ-ㅇㅇ-2019-00000')

2500220041486130993

In [16]:
hash('ㅇㅇ-ㅇㅇ-2019-00001')

2306157997212821217

In [17]:
hash('ㅇㅇ-ㅇㅇ-2019-00002')

-6216469757349641193

In [18]:
# abs 함수를 쓰면 절대값을 구할 수 있습니다.

abs(hash('ㅇㅇ-ㅇㅇ-2019-00002'))

6216469757349641193

## DB 접속

In [19]:
# 파일로 기록되는 데이터베이스에 연결/접속

connection = sqlite3.connect('test.db')

In [20]:
# 커서 얻기

cursor = connection.cursor()

## 테이블 생성

In [21]:
sql = '''
CREATE TABLE animals (
    id INTEGER PRIMARY KEY AUTOINCREMENT, #hash를 썼기 때문에 우리가 넣은 값이(id) 들어감 ㅎㅎ 그래서 중복 안됨 
    age TEXT,
    colorCd TEXT,
    happenDt TEXT,
    kindCd TEXT,
    orgNm TEXT,
    sexCd TEXT,
    weight TEXT
)
'''
# 스키마 라고 부름 (위의 sql 테이블)
# 두번 실행하고 싶을 때? drop table animals 추가하면 되는 데, 한번 더 실행하면 덮여서 사라짐..! 정말 2번 실행해야하는 경우만 하기 
cursor.execute(sql)

OperationalError: table animals already exists

## XML 파싱

In [22]:
# XML 데이터를 rows에 담기

# rows 준비
rows = []

for item in root.iter('item'):
    # row 준비
    row = {}

    # 중복을 방지하기 위해 noticeNo 컬럼을 숫자로 바꿔서 Identifier로 활용합니다.
    # 문자열을 숫자로 바꿀 땐 hash 함수를 사용합니다.
    row['id'] = abs(hash(item.find('noticeNo').text))

    # 원하는 컬럼만 얻습니다.
    for column in columns:
        row[column] = item.find(column).text

    # rows에 추가
    rows.append(row)

In [23]:
# rows 갯수 확인

len(rows)

100

In [24]:
# 데이터 확인

rows[0]

{'id': 7397766572298055606,
 'age': '2019(년생)',
 'colorCd': '검정',
 'happenDt': '20191023',
 'kindCd': '[개] 믹스견',
 'orgNm': '제주특별자치도',
 'sexCd': 'M',
 'weight': '2(Kg)'}

## DB에 데이터 넣기

Identifier로 쓰이는 `id`를 제외한 나머지 값을 `INSERT`해야 합니다.

`INSERT INTO [테이블 이름] ([필드 목록]) VALUES ([값 목록])`

필드 목록을 만들기 위해 우리는 `join`을 사용할 겁니다.

- [str.join](https://github.com/ahastudio/til/blob/master/python/str-join.md)
- [SQLite Python: Inserting Data](https://www.sqlitetutorial.net/sqlite-python/insert/)

In [25]:
# 데이터의 키를 모아서 쉼표로 이어줍니다.

', '.join(rows[0].keys()) #rows 0번 리스트가 담고 있는 키 

'id, age, colorCd, happenDt, kindCd, orgNm, sexCd, weight'

In [26]:
# 필드 목록에 fields란 이름을 붙입니다.

fields = ', '.join(rows[0].keys()) #,로 조인한다 

# f-string을 사용하면 중괄호({}) 안에 변수 등을 넣어서 아주 간단히 SQL을 만들 수 있습니다.

sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'

print(sql)

INSERT INTO animals (id, age, colorCd, happenDt, kindCd, orgNm, sexCd, weight) VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [27]:
# 테이블에 데이터 넣기

# 만약 id가 중복되면 “IntegrityError: UNIQUE constraint failed: animals.id” 에러가 발생합니다.

for row in rows:
    fields = ', '.join(row.keys())
    sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(sql, list(row.values()))

connection.commit()

In [28]:
# 중복을 피해서 테이블에 데이터 넣기

for row in rows[:3]:
    fields = ', '.join(row.keys())
    sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'
    try:
        cursor.execute(sql, list(row.values()))
    except sqlite3.IntegrityError:
        print('중복된 데이터:', row)

connection.commit()

중복된 데이터: {'id': 7397766572298055606, 'age': '2019(년생)', 'colorCd': '검정', 'happenDt': '20191023', 'kindCd': '[개] 믹스견', 'orgNm': '제주특별자치도', 'sexCd': 'M', 'weight': '2(Kg)'}
중복된 데이터: {'id': 9129314992545460286, 'age': '2018(년생)', 'colorCd': '흰색,검정', 'happenDt': '20191023', 'kindCd': '[개] 보스턴 테리어', 'orgNm': '경상남도 거제시', 'sexCd': 'M', 'weight': '10(Kg)'}
중복된 데이터: {'id': 1312205796456516392, 'age': '2017(년생)', 'colorCd': '황색 ', 'happenDt': '20191023', 'kindCd': '[개] 믹스견', 'orgNm': '경상남도 밀양시', 'sexCd': 'F', 'weight': '8(Kg)'}


## DB에 데이터가 잘 들어갔는지 간단히 확인

In [29]:
# 테이블에서 데이터 얻기

cursor.execute('SELECT * FROM animals')

rows = cursor.fetchall()

In [30]:
# 전부 출력하면 무서울테니 갯수만 확인합니다.

print(len(rows))

30729


In [32]:
# 데이터 하나만 확인합니다.

print(rows[0])

(1, '2016(년생)', '흰갈', '20191014', '[개] 믹스견', '경상남도 의령군', 'F', '5(Kg)')


## DB 사용 종료

DB를 사용 후엔 `close`를 하는 게 좋습니다.

In [33]:
connection.close()